# Проект 2: Исследование надёжности заёмщиков

<br/>
    
<div class="alert alert-info">
<h2> Привет!<a class="tocSkip"></h2>
<hr>  
Я - Сергей. Рад что ты читаешь это текст :) Сделаю пару ремарок общего характера. В описании выполняемых действий, комментариях, я использую местоимение 'мы', так как считаю что ты, читая код, становиться моим соучастником ('соучавствующие программирование') и мы вместе проходим по пути исследования. Да и сформировлась привычка так писать в ходе написания научных работ.   
NB! - так помечаю важные на мой взгдял идеи, которые стоит взять на заметку; инсайты, проработка которых требует большего знания предметной области.   
Надеюсь мои многословные комментарии тебя не утомят.    
</div>
    
<br/>

### Описание проекта     
**Заказчик** — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.  
**Входные данные от банка** — статистика о платёжеспособности клиентов.  
Результаты исследования будут учтены при построении модели *кредитного скоринга* — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.  
**Описание данных**  
children — количество детей в семье  
days_employed — общий трудовой стаж в днях  
dob_years — возраст клиента в годах  
education — уровень образования клиента  
education_id — идентификатор уровня образования  
family_status — семейное положение  
family_status_id — идентификатор семейного положения  
gender — пол клиента  
income_type — тип занятости  
debt — имел ли задолженность по возврату кредитов  
total_income — ежемесячный доход  
purpose — цель получения кредита  

### Оглавление  
0.1. [Шаг 1. Обзор данных](#step_1)  
0.2. [Шаг 2.1 Заполнение пропусков](#step_2_1)  
0.3. [Шаг 2.2 Проверка данных на аномалии и исправления](#step_2_2)   
0.4. [Шаг 2.3. Изменение типов данных](#step_2_3)   
0.5  [Шаг 2.4. Удаление дубликатов](#step_2_4)   
0.6  [Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма](#step_2_5)  
0.7  [Шаг 2.6. Категоризация дохода](#step_2_6)   
0.8  [Шаг 2.7. Категоризация целей кредита](#step_2_7)    
0.9  [Ответы на вопросы](#answers)  
1  [Общий вывод](#end)    

### Шаг 1. Обзор данных <a id='step_1'></a>

In [1]:
# Имортируем библиотеку pandas с сокращённым именем pd
import pandas as pd

In [2]:
# Прочитаем файл с гугл диска
from io import BytesIO
import requests
spreadsheet_id = '1ifkGj2NBfVQk13MvxAWcH-Pyoqjt2-8hu7sXxFuKafA'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
data = pd.read_csv(BytesIO(r.content))

***Выводы из обзора данных:***  
1. в таблице 21525 строк (наблюдений) и 12 колонок (атрибутов);
1. в двух колонках 'days_employed' (стаж в днях) и 'total_income' (ежемесячный доход) есть пропущенные значения;  
1. в колонке 'days_employed' есть как положительные, так и отрицательные значения - аномалия в данных;
1. 2-х колонках хранятся данные типа float64 числа с плавающей точкой - числа с дробной частью); 
1. 5-и колонках хранятся данные типа int64 (целые чиcла); 
1. 5-и колонках хранятся данные типа object (текстовые или смешанные числовые и нечисловые значения);
1. колонки названы корректно: заголовки написаны в 'змеинном_стиле', в них нет лишних пробелов, заглавных букв.

### Шаг 2.1 Заполнение пропусков<a id='step_2_1'></a>

Обзор данных показал, что в двух колонках 'days_employed' (колонка с числовыми данными) и 'total_income' (колонка с числовыми данными) есть пропущенные значения. Далее узнаем число пропусков и их долю от общего числа значений в колонках. После найдём их и заполним медианным значением по столбцу. Также рассмотрим причины пояления пропусков и объясним почему применяется медиана для заполения пропусков.

In [3]:
loss_days_employed = len(data[data['days_employed'].isna()])# переменная с числом строк с пропусками в колонке 'days_employed'
loss_total_income = len(data[data['total_income'].isna()])# переменная с числом строк с пропусками в колонке 'total_income'
# Печать числа строк с пропусками
print('Число пропущеных значений в колонке "days_employed":', loss_days_employed)
print('Число пропущеных значений в колонке "total_income":', loss_total_income)
# печать доли строк с пропусками от общего числа строк, 
# с расчётом показателя внутри функции print, с представлением в виде %-в (с точнотью до ноль знака после запятой)
print('Доля пропущеных значений в колонке"days_employed": {:.0%}'.format(loss_days_employed/len(data['days_employed'])))
print('Доля пропущеных значений в колонке"total_income": {:.0%}'.format(loss_total_income/len(data['total_income'])))
# в колонках 'days_employed' и 'total_income' пропущено каждое десятое значение.

Число пропущеных значений в колонке "days_employed": 2174
Число пропущеных значений в колонке "total_income": 2174
Доля пропущеных значений в колонке"days_employed": 10%
Доля пропущеных значений в колонке"total_income": 10%


In [4]:
# Заменим пропущенные значения медианным значением (медианой)
#days_employed_median = data['days_employed'].median() #расчёт медианы для колонки 'days_employed'.
#total_income_median = data['total_income'].median() #расчёт медианы для колонки 'total_income'.
# NB! Медианные значения я бы посчитал с учётом ввозраста клиента для стажа и типа занятости для дохода.
# найдём пропущенные значения и заполним их медианным значением по столбцу
#data['days_employed'] = data['days_employed'].fillna(value = days_employed_median)
#data['total_income'] = data['total_income'].fillna(value = total_income_median)
#print(days_employed_median) # напечатаем медиану колонки 'days_employed'
#print(total_income_median) # напечатаем медиану колонки 'total_income'
#data.info()# вызовем метод info() для получения информации об обновленном датасете
# пропущеных значений в колонках больше нет

In [5]:
# Чтобы заполнить пропущенные значения в столбце трудовой стаж (days_employed) в днях 
#можно взять медиану для каждой группы возраста. Для этого создадим новый столбец "age_group"
# 1 группа: 19 - 30
# 2 группа: 30 - 40
# 3  группа: 40 - 55 
# 4  группа: 55 - 75

def days_employed(row):
    
    age = row['dob_years']

    if age <= 30:
        return '1 группа'
    
    if age <= 40 and age > 30:
        return '2 группа'
    
    if age <= 55 and age > 40:
        return '3 группа'
    
    if age <= 80 and age > 55:
        return '4 группа'

data['age_group'] = data.apply(days_employed, axis=1)

data['days_employed'] = data['days_employed'].fillna(0)
data.isnull().sum()

data.groupby('age_group')['days_employed'].mean()[0]

data.loc[(data['age_group'] == '1 группа') & (data['days_employed'] == 0 ), 'days_employed'] = data.groupby('age_group')['days_employed'].mean()[0]
data.loc[(data['age_group'] == '2 группа') & (data['days_employed'] == 0 ), 'days_employed'] = data.groupby('age_group')['days_employed'].mean()[1]
data.loc[(data['age_group'] == '3 группа') & (data['days_employed'] == 0 ), 'days_employed'] = data.groupby('age_group')['days_employed'].mean()[2]
data.loc[(data['age_group'] == '4 группа') & (data['days_employed'] == 0 ), 'days_employed'] = data.groupby('age_group')['days_employed'].mean()[3]

In [6]:
# Чтобы заполнить пропущенные значения в столбце Total Income можно использовать медиану дохода 
# для каждого типа дохода (income_type)

data['total_income'] = data['total_income'].fillna(0)

data.groupby('income_type')['total_income'].median()

data.loc[(data['total_income'] == 0) & (data['income_type'] == 'безработный'), 'total_income'] = data.groupby('income_type')['total_income'].median()[0]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'в декрете'), 'total_income'] = data.groupby('income_type')['total_income'].median()[1]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'госслужащий'), 'total_income'] = data.groupby('income_type')['total_income'].median()[2]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'компаньон'), 'total_income'] = data.groupby('income_type')['total_income'].median()[3]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'пенсионер'), 'total_income'] = data.groupby('income_type')['total_income'].median()[4]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'предприниматель'), 'total_income'] = data.groupby('income_type')['total_income'].median()[5]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'сотрудник'), 'total_income'] = data.groupby('income_type')['total_income'].median()[6]
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'студент'), 'total_income'] = data.groupby('income_type')['total_income'].median()[7]

data.info()# вызовем метод info() для получения информации об обновленном датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  age_group         21525 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


**Вывод 1:** В колонках 'days_employed' и 'total_income' пропущено каждое десятое значение.  
**Вывод 2:** Колонка 'days_employed' содержит значения об общем трудовом стаже потенциального заёмщика в днях, а 'total_income' данные ежемесячном доходе - остутвие данных (пропуски) может свидетельствовать о:  
во-перых, об ошибке при вводе данных сотрудником;   
во-вторых, клиент не заполнил их в анкете (форме) - не хотел раскрывать, не вспомнил (особенно стаж в днях), пропустил;  
в-третих, об отсутвие у клиента работы в данным момент и в прошлом. Клиент или ещё не достиг трудоспосбного возраста, или в данным момент безработный (или занят, но нефоциально) и ни когда не работал официально.  
**Вывод 3:** В первых двух случаях использование для заполнения пропусков медианного значения оправдано тем, что в отличии от среднего арифметического, медиана не чувствительно к выбросам - слишком низким или высоким значеним. При условии что мы не можим удалить наблюдение (клиента) или применять более сложные методы - медиана лучший вариант.     
Есть разные мнения по поводу использование использование медианы и других методов для заполнения пропусков (https://loginom.ru/blog/missing), но эти методы не всегда приводят к ухудшению результатов. Более того, когда это уместно, использование простых методов (такого как - заполнение медианой) более предпочтительно.   
**NB!** В третьем случае без дополнительного анализа, сбора дополнительных данных о клиенте - путём его интервью, например, я бы не стал использовать среднее, это улучшит искусственно рейтинг клиента для банка и скроет потенциальный риск невозврата.

### Шаг 2.2 Проверка данных на аномалии и исправления. <a id='step_2_2'></a>

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке.

In [7]:
# Посмотрим на список уникальных значений в каждой колонке.
# что бы не выводить каждую колонку в ручную, переберём их имена в цикле и выведем уникальные значения
data_columns = ['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose']
for elem in data_columns:
    print('имя колонки', elem, 'значения', data[elem].unique())

имя колонки children значения [ 1  0  3  2 -1  4 20  5]
имя колонки days_employed значения [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]
имя колонки dob_years значения [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
имя колонки education значения ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
имя колонки education_id значения [0 1 2 3 4]
имя колонки family_status значения ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
имя колонки family_status_id значения [0 1 2 3 4]
имя колонки gender значения ['F' 'M' 'XNA']
имя колонки income_type значения ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безра

In [8]:
#В колонке 'total_income'уникальных значений много,
# но доход не может быть отрицательным,
display(data['total_income'].sort_values().head())# поэтому отсоритуем значения по возрастанию 
# и посмотрим на 5 самых низких значений.

14585    20667.263793
13006    21205.280566
16174    21367.648356
1598     21695.101789
14276    21895.614355
Name: total_income, dtype: float64

**Результаты проверки колонок и возможные причины возниконовения аномалий:**   
1. колонка 'children' - отрицательное число детей (-1), есть значение 20 детей - это в теории возможно, но скорее аномалия. Заменим на 1 и 2 соответственно. Причина аномалий - опечатки сотрудников вводивших данные;   
1. колонка 'days_employed' - отрицательное число дней стажа. Причина аномалий - опечатки сотрудников вводивших данные, непонимание того как правильно вводить данные;  
1. колонка 'dob_years' - возраст клиента (-ов) равный нулю. Причина аномалий - отсутвие данных;  
1. колонка 'education' - одни и те же значения записаны по разному: только строчными буквами, только заглавными, с заглавной буквы строчными. Причина аномалий - нет единого стандарста записи; 
1. колонка 'education_id' - аномалий нет;  
1. колонка 'family_status' - одни и те же значения записаны по разному: только строчными буквами, с заглавной буквы строчными. Причина аномалий - нет единого стандарта записи; 
1. колонка 'family_status_id' - аномалий нет;  
1. колонка 'gender' - странное значение 'XNA'. Причина аномалий - опечатки сотрудников вводивших данные; 
1. колонка 'income_type' - аномалий нет;  
1. колонка 'debt' - аномалий нет;
1. колонка 'total_income' - аномалий нет;
1. колонка 'purpose' - часть значений можно привести к общей формулировке, уменьшив таким образом их число и стандартизировав формулировки. Причина аномалий - нет единого стандарта записи. 

Далее мы последовательно устраним аномалии в каждой из колонок, где они есть.

In [9]:
# Колонка children, заменим значения -1 и 20 на 1 и 2
data['children'] = data['children'].replace(20, 2)
data['children'] = data['children'].replace(-1, 1)
#data['children'].unique()# закомментировал строку чтобы её вывод не мешал дальнешим шагам

In [10]:
# Колонка days_employed содержит отрицательные значения.
# Возьмем значения колонки по модулю и избавимся от отрицательных значений. 
#Решение нагуглил https://pythonru.com/osnovy/abs-v-python-modul-chisla
data['days_employed'] = abs(data['days_employed'])
#display(data.head())

In [11]:
# Колонка dob_years содержит значения возраста равное нулю, узнаем количество наблюдений с таким значеним
#data[data['dob_years'] == 0].count() #считаем число нулевых значений, закомментировал строку чтобы её вывод не мешал дальнешим шагам
#Всего 101 нулевое значение.
# Для того чтобы заполнить нулевые значения
#посчитаем медиану возраста клиента в годах  для каждого типа занятости
#Гипотеза: медианный возраст клиентов с типом занятости 'пенсионер', например, 
#должен быть выше аналогичного показателя для клиентов с типом занятости 'студент'
income_type_median_years = data.groupby('income_type')['dob_years'].median()
display(income_type_median_years) #выведем результат
# Заполним нулевые значения в колонке медианными значениями для каждого типа занятости
data.loc[(data['income_type'] == 'безработный ') & (data['dob_years'] == 0), 'dob_years'] = 38
data.loc[(data['income_type'] == 'в декрете') & (data['dob_years'] == 0), 'dob_years'] = 39
data.loc[(data['income_type'] == 'госслужащий') & (data['dob_years'] == 0), 'dob_years'] = 40
data.loc[(data['income_type'] == 'компаньон') & (data['dob_years'] == 0), 'dob_years'] = 39
data.loc[(data['income_type'] == 'пенсионер') & (data['dob_years'] == 0), 'dob_years'] = 60
data.loc[(data['income_type'] == 'предприниматель') & (data['dob_years'] == 0), 'dob_years'] = 42.5
data.loc[(data['income_type'] == 'сотрудник') & (data['dob_years'] == 0), 'dob_years'] = 39
data.loc[(data['income_type'] == 'студент') & (data['dob_years'] == 0), 'dob_years'] = 22
#data[data['dob_years'] == 0].count() #считаем число нулевых значений - их нет. 
# закомментировал строку чтобы её вывод не мешал дальнешим шагам

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

In [12]:
# Колонка education, значения в разном регистре. Приведем  значения к единому (строчному) регистру
data['education'] = data['education'].str.lower()
data['education'].unique()
# в колонке education всего 5 уникальных значений

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [13]:
# Колонка family_status, значения в разном регистре. Приведем  значения к единому (строчному) регистру
data['family_status'] = data['family_status'].str.lower()
data['family_status'].unique()
# в колонке education всего 5 уникальных значений

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [14]:
#  Колонка gender содержит значение непонятное 'XNA'. Узнаем количество пропущенных значений
data[data['gender'] == 'XNA'].count()
# значение 'XNA' встречается один раз, исключим это наблюдение из выборки.
data = data[data['gender'] != 'XNA']
#data['gender'].unique() # закомментировал строку чтобы её вывод не мешал дальнешим шагам

### Шаг 2.3. Изменение типов данных.<a id='step_2_3'></a>

In [15]:
# Вещественные значения данных в двух колонках days_employed и total_income.
# заменим их на цельночисленные (тип 'int') методом astype()
data['days_employed'] = data['days_employed'].astype('int') # изменим значения колонки days_employed в тип 'int'
data['total_income'] = data['total_income'].astype('int') # изменим значения колонки total_income в тип 'int'
#display(data.head()) # закомментировал строку чтобы её вывод не мешал дальнешим шагам

### Шаг 2.4. Удаление дубликатов.<a id='step_2_4'></a>

Подсчитаем число дублирующих друг-друг наблюдений (строк), при этом основываемся на предположении что запись где совпадают значения во всех колонках - это запись об одном клиенте.   
Дублирование записей может также негативно сказать на скорринговой модели - изказив её выводы, вызыва переобучение модели.   
Ранее, значения в ряде колонок мы уже привели к единому регистру - это исключит неявные дубликаты.

In [16]:
# Подсчитаем число дублирующих друг-друг наюлюдений.
# В базе 71 дублирующая запись, исключим эти наблюдения из выборки.
data = data.drop_duplicates().reset_index(drop=True) #удаление дублирующих записей.
data.duplicated().sum() #а теперь ноль

0

**Причины появления дубликатов:**  
* естественное дублирование строк - данные об одном клиенте могли внести дважды в ходя текущей работы, например если он дважны подавал (заполнял) заявку на кредит;  
* случайное дублирование - по техническим причинам (случайно) произошло дублирование записей;  
* искусственное дублирование строк - кто-то преднамерено, для каких-то целей, задублировал строки. 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.<a id='step_2_5'></a>

Создадим два новых дадафрейма:   
1. education, куда включим колонки 'education' и 'education_id';  
1. family_status, куда включим колонки 'family_status' и 'family_status_id' из датафрейма data.  

Удалим из датафрейма data колонки 'education' и 'family_status', оставив только их идентификаторы 'education_id' и 'family_status_id'.

In [17]:
education = data[['education', 'education_id']].drop_duplicates().reset_index(drop=True) # создаём датафрейм education
# и удалим из него дубликаты
family_status = data[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True) 
#создаём датафрейм family_status и удалим из него дубликаты
display(education.head()) # посмотрим на новые датафреймы, закомментировал строку чтобы её вывод не мешал дальнешим шагам
display(family_status.head()) # закомментировал строку чтобы её вывод не мешал дальнешим шагам

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [18]:
# Удалим из датафрейма data колонки 'education' и 'family_status' методом drop().
data = data.drop('education', 1)
data = data.drop('family_status', 1)
display(data.head()) # выведем на экран обновлённый датафрейм

C:\Users\Sergey\AppData\Local\Temp/ipykernel_8272/1450935768.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('education', 1)
C:\Users\Sergey\AppData\Local\Temp/ipykernel_8272/1450935768.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('family_status', 1)


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,3 группа
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,2 группа
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,2 группа
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,2 группа
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,3 группа


### Шаг 2.6. Категоризация дохода.<a id='step_2_6'></a>

Создадим функцию, которая на основании данных из колонки total_income будет присваивать клиентам категорию их дохода и записывать в новой колонке total_income_category.  
Категории доходов будут присавиваться на основании диапазонов, указанных ниже:  
* 0–30000 — 'E';  
* 30001–50000 — 'D';  
* 50001–200000 — 'C';  
* 200001–1000000 — 'B';  
* 1000001 и выше — 'A'.  

In [19]:
def total_income_category(total_income): # создаём функцию
    try:
        if total_income <= 30000: # проверим доход на соответсвие критерию категории Е, и далее каждой из категорий
            return 'E'
    
        if total_income >= 30001 and total_income <= 50000:
            return 'D'
    
        if total_income >= 50001 and total_income <= 200000:
            return 'C'
    
        if total_income >= 200001 and total_income <= 1000000:
            return 'B'
    
        if total_income >= 1000001:
            return 'A'
    except:
        print('Не получилось присвоить категорию')
# Создаём колонку total_income_category с категориями дохода, и в её ячейках записиваем значения, возвращаемые функцией
data['total_income_category'] = data['total_income'].apply(total_income_category)
display(data.head(10)) 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,3 группа,B
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,2 группа,C
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,2 группа,C
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,2 группа,B
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,3 группа,C
5,0,926,27.0,0,1,M,компаньон,0,255763,покупка жилья,1 группа,B
6,0,2879,43.0,0,0,F,компаньон,0,240525,операции с жильем,3 группа,B
7,0,152,50.0,1,0,M,сотрудник,0,135823,образование,3 группа,C
8,2,6929,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы,2 группа,C
9,0,2188,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи,3 группа,C


### Шаг 2.7. Категоризация целей кредита.<a id='step_2_7'></a>

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.

In [20]:
def purpose_category(purpose): #создаём функцию, которая будет проверять есть ли в строках те или иные леммы,
    # и соответственно им присваивал цель кредита
    try:
        if 'автомоб' in purpose:
            return 'операции с автомобилем'
    
        if 'жиль' in purpose:
            return 'операции с недвижимостью'
    
        if 'недвиж' in purpose:
            return 'операции с недвижимостью'
    
        if 'свадь' in purpose:
            return 'проведение свадьбы'
    
        if 'образов' in purpose:
            return 'получение образования'
    except:
        print('Не получилось присвоить категорию')
# Создаём колонку purpose_categoryс категориями дохода, и в её ячейках записиваем значения, возвращаемые функцией
data['purpose_category'] = data['purpose'].apply(purpose_category)
display(data.head(100)) 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_category,purpose_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,3 группа,B,операции с недвижимостью
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,2 группа,C,операции с автомобилем
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,2 группа,C,операции с недвижимостью
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,2 группа,B,получение образования
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,3 группа,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,541,51.0,0,0,F,сотрудник,0,94187,автомобиль,3 группа,C,операции с автомобилем
96,0,34261,44.0,1,0,F,сотрудник,0,133546,покупка жилой недвижимости,3 группа,C,операции с недвижимостью
97,0,34261,47.0,0,0,F,сотрудник,0,133546,профильное образование,3 группа,C,получение образования
98,0,364906,54.0,0,0,F,пенсионер,0,199707,покупка жилья для сдачи,3 группа,C,операции с недвижимостью


### Ответы на вопросы.<a id='answers'></a>

***Данные очищенны и их можно сгрупировать их для того чтобы ответить на поставленые в задании вопросы:***  
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
1. Есть ли зависимость между семейным положением и возвратом кредита в срок?
1. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
1. Как разные цели кредита влияют на его возврат в срок?  

А для начала, посчитаем долю кредитов, платежи по которым были просрочены в целом по всему датафрейму;  

In [21]:
# Посчитаем долю кредитов, платежи по которым были просрочены в целом по всем наюлюдениям
late_payment = data[(data['debt'] == 1)].count() / data[(data['debt'] == 0)].count()# 1 - просрочка есть, 0 - просрочник нет
#late_payment['debt'] # закомментировал строку чтобы её вывод не мешал дальнешим шагам
print('В целом, кредиты не выплачивается во время в {:.1%} случаев'.format(late_payment['debt']))

В целом, кредиты не выплачивается во время в 8.8% случаев


##### Вопрос 1: Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
data_pivot_child = data.pivot_table(index=['children'], columns='debt', values='family_status_id', aggfunc='count')
# Создадим в сводной таблице новую колонку late_payment_percent 
# и сохраните в нём значение отношение просроченных кредитов к обшему числу кредитов - 
# сумме просроченых и непросроченых, тем самым узнаем долю просрочненых в разрезе числа детей
data_pivot_child['late_payment_percent'] = (data_pivot_child[1] / (data_pivot_child[1] + data_pivot_child[0]))*100
display(data_pivot_child)
# У семей с 5 детьми значение NaN, узнаем почему. 
# Методом unique() узнаем какие уникальные значения для семей с 5 детьми есть в колонке debt. 
data.loc[data.loc[:,'children'] == 5]['debt'].unique() # в колонке debt у семей с 5 детьми
# только одно уникальное значение 0 - у них нет просрочек.

debt,0,1,late_payment_percent
children,,,
0,13027.0,1063.0,7.544358
1,4410.0,445.0,9.165808
2,1926.0,202.0,9.492481
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


array([0], dtype=int64)

**Вывод 1:** Появляние первого и каждого последующего ребёнка увеличивает возможность просрочки платежа по кредиту.   
Семьям с 5-ю детьми не допусками просрочек - но, выборка по таким семьям слишком маленькая, всего 9 наблюдений.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
data_pivot_family_status = data.pivot_table(index=['family_status_id'], columns='debt', values ='children', aggfunc='count')
# Создадим в сводной таблице новую колонку late_payment_percent 
# и сохраните в нём значение отношение просроченных кредитов к обшему числу кредитов - 
# сумме просроченых и непросроченых, тем самым узнаем долю просрочненых в разрезе семейного положения
data_pivot_family_status['late_payment_percent'] = (data_pivot_family_status[1] / (data_pivot_family_status[1] + data_pivot_family_status[0]))*100
# и методом merge() подтянем в сводную таблицу расшифровку семейных статусов
data_pivot_family_status = data_pivot_family_status.merge(family_status, on='family_status_id', how='left')
display(data_pivot_family_status)

,family_status_id,0,1,late_payment_percent,family_status
0,0,11408,931,7.545182,женат / замужем
1,1,3762,388,9.349398,гражданский брак
2,2,896,63,6.569343,вдовец / вдова
3,3,1110,85,7.112971,в разводе
4,4,2536,274,9.750890,не женат / не замужем


**Вывод 2:** Не женатые/ не замужние или люди в гражданском браке реже выплачивают кредит в срок.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
data_pivot_income_category = data.pivot_table(index=['total_income_category'], columns='debt', values ='family_status_id', aggfunc='count')
# Создадим в сводной таблице новую колонку late_payment_percent 
# и сохраните в нём значение отношение просроченных кредитов к обшему числу кредитов - 
# сумме просроченых и непросроченых, тем самым узнаем долю просрочненых в разрезе категорий дохода
data_pivot_income_category['late_payment_percent'] = (data_pivot_income_category[1] #код длинный перенесём на новую строку
                                                      / (data_pivot_income_category[1] + data_pivot_income_category[0]))*100
display(data_pivot_income_category)

debt,0,1,late_payment_percent
total_income_category,,,
A,23,2,8.000000
B,4685,356,7.062091
C,14655,1360,8.492039
D,329,21,6.000000
E,20,2,9.090909


**Вывод 3:** Чаще всего кредиты вовремя не выплачивают заёмщики с низким доходом - категория Е (чаще чем в среднем по всем заёмщикам в целом), но выборка по этим заёмщикам небольшая, и заёмщики со средном доходом - это самая массовая категория заёмщиков.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [25]:
data_pivot_purpose_category = data.pivot_table(index=['purpose_category'], columns='debt', values='family_status_id', aggfunc='count')
# Создадим в сводной таблице новую колонку late_payment_percent 
# и сохраните в нём значение отношение просроченных кредитов к обшему числу кредитов - 
# сумме просроченых и непросроченых, тем самым узнаем долю просрочненых в разрезе целей кредитов
data_pivot_purpose_category['late_payment_percent'] = (data_pivot_purpose_category[1] #код длинный перенесём на новую строку
                                                       / (data_pivot_purpose_category[1] + data_pivot_purpose_category[0]))*100
display(data_pivot_purpose_category)

debt,0,1,late_payment_percent
purpose_category,,,
операции с автомобилем,3903,403,9.359034
операции с недвижимостью,10028,782,7.234043
получение образования,3643,370,9.220035
проведение свадьбы,2138,186,8.003442


**Вывод 4:** Заёмщики чаще выплачивают кредиты, взятые на операции недвижимостью или на свадьбу. 


## Общий вывод:<a id='end'></a>

При работе с данными были заполнены пропущенные значения, устранены аномальные значения, удалены дубликаты. С помощью функций были привоены категории дохода заёмщиков и целей получения кредита. Применив сводные таблицы произведена оценка того, какие критерии влияют на возврат кредита.  
* Появляние первого и каждого последующего ребёнка увеличивает возможность просрочки платежа по кредиту.  
* Не женатые/ не замужние или люди в гражданском браке реже выплачивают кредит в срок.   
* Чаще всего кредиты вовремя не выплачивают заёмщики с низким доходом (чаще чем в среднем по всем заёмщикам в целом), и заёмщики со средном доходом - но, это самая массовая категория заёмщиков.   
* Заёмщики чаще выплачивают кредиты, взятые на операции недвижимостью или на свадьбу.   

Дополнительный анализ может помочь более точно предсказать вероятность возврата кредита в срок.